In [1]:
# !pip install pycaret

In [2]:
%cd D:/GitHub/machine-learning/mlops/Alumni-Sessions-main/Alumni-Sessions-main/MLOps/Session01/project/mlops_experiment/

D:\GitHub\machine-learning\mlops\Alumni-Sessions-main\Alumni-Sessions-main\MLOps\Session01\project\mlops_experiment


In [3]:
from src.features import helper_features

In [4]:
import pandas as pd

In [5]:
data_path = "/Users/lavi/Documents/Consulting Work/Upgrad/Alumni Sessions/MLOps/Session01/project/mlops_experiment/data/raw/"
dataset = pd.read_csv(data_path+"WA_Fn-UseC_-Telco-Customer-Churn.csv")
dataset.head(3)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/lavi/Documents/Consulting Work/Upgrad/Alumni Sessions/MLOps/Session01/project/mlops_experiment/data/raw/WA_Fn-UseC_-Telco-Customer-Churn.csv'

In [ ]:
import pandas as pd
from pandas_profiling import ProfileReport
profile = ProfileReport(dataset, title="Pandas Profiling Report", explorative=True)

In [ ]:
report_path = "/Users/lavi/Documents/Consulting Work/Upgrad/Alumni Sessions/MLOps/Session01/project/mlops_experiment/reports/"
profile.to_file(report_path+"data_report_01.html")

In [ ]:
helper_features.dataset_info(dataset, "customers")

In [ ]:
helper_features.pie_plot(dataset["Churn"].value_counts(), plot_title="<b>Client Churn Distribution<b>")

In [ ]:
df_aux = dataset.query('Churn == "Yes"')
df_aux = df_aux["Contract"].value_counts()
helper_features.bar_plot(df_aux, "<b>Contract Types of Churned Clients</b>", "<i>Contract</i>", "<i>Count</i>")

In [ ]:
df_aux = dataset.query('(Contract == "Month-to-month") and (Churn == "No")')
helper_features.histogram_subplot(df_aux, df_aux, "MonthlyCharges", "TotalCharges", "<b>Charges Distribution for Month-to-month contracts for not Churned Clients</b>",
                "(a) Monthtly Charges Distribution", "(b) Total Charges Distribution")

In [ ]:
df_aux = dataset.query('(Contract == "Month-to-month") and (Churn == "Yes")')
helper_features.histogram_subplot(df_aux, df_aux, "MonthlyCharges", "TotalCharges", "<b>Charges Distribution for Month-to-month contracts for Churned Clients</b>",
                "(a) Monthtly Charges Distribution", "(b) Total Charges Distribution")

In [ ]:
df_aux = dataset.query(('Contract == Month-to-month') and ('Churn == "Yes"'))
df_aux = df_aux["PaymentMethod"].value_counts()
helper_features.bar_plot(df_aux, "<b>Payment Method of Month-to-month contract Churned Clients</b>", "<i>Payment Method</i>", "<i>Count</i>")

In [ ]:
df_aux = dataset.query(('Contract == Month-to-month') and ('Churn == "Yes"'))
df_aux = df_aux["tenure"].value_counts().head(5)
helper_features.bar_plot(df_aux, "<b>Tenure of Month-to-month contract for Churned Clients</b>", "<i>Tenure</i>", "<i>Count</i>")

In [ ]:
dataset['tenure'] = dataset['tenure'].astype(float)
dataset['MonthlyCharges'] = dataset['MonthlyCharges'].astype(float)
dataset['TotalCharges'] = pd.to_numeric(dataset['TotalCharges'],errors='coerce')

In [ ]:
data, data_unseen = helper_features.data_sampling(dataset, 0.9, random_seed=123)
print(f"There are {data_unseen.shape[0]} samples for Unseen Data.")

In [ ]:
from pycaret.classification import *

In [ ]:
exp01 = setup(data=data, target="Churn", session_id=123, ignore_features=["customerID"], 
                numeric_features=["SeniorCitizen"],normalize=True, 
              feature_selection=True,log_experiment = True, experiment_name = 'churn_02',
              log_plots=True, log_profile=True)

In [ ]:
compare_models(fold=10, sort="F1")

In [ ]:
base_alg = "lr"
base_model = create_model(base_alg)

In [ ]:
plot_model(base_model, plot="parameter")

In [ ]:
plot_model(base_model)

In [ ]:
plot_model(base_model, plot="pr")

In [ ]:
plot_model(base_model, plot="confusion_matrix")

In [ ]:
plot_model(base_model, plot="error")

In [ ]:
plot_model(base_model, plot="class_report")

In [ ]:
plot_model(base_model, plot="boundary")

In [ ]:
plot_model(base_model, plot="feature")

In [ ]:
plot_model(base_model, plot="calibration")

In [ ]:
tuned_model = tune_model(base_model, fold=10, optimize="F1")

In [ ]:
plot_model(tuned_model, plot="parameter")

In [ ]:
plot_model(tuned_model, plot="confusion_matrix")

In [ ]:
plot_model(tuned_model, plot="feature")

In [ ]:
bagged_model = ensemble_model(tuned_model)

In [ ]:
print(bagged_model)

In [ ]:
boosted_dt = ensemble_model(bagged_model, method = 'Boosting')

In [ ]:
blended_model = blend_models(estimator_list=[tuned_model, bagged_model], method="soft")

In [ ]:
blended_model = blend_models(estimator_list=[tuned_model, bagged_model], method="hard")

In [ ]:
nb = create_model('nb')
lr = create_model('lr')
lda = create_model('lda')

In [ ]:
blend_soft = blend_models(estimator_list = [nb, lda, lr], method = 'soft')

In [ ]:
top5 = compare_models(n_select = 5)

In [ ]:
stack_soft = stack_models(top5)

##Model Calibration 

When performing classification you often not only want to predict the class label (outcome such as 0 or 1), but also obtain the probability of the respective outcome which provides a level of confidence on the prediction. Some models can give you poor estimates of the class probabilities and some do not even support probability prediction. Well calibrated classifiers are probabilistic and provide outputs in the form of probabilities that can be directly interpreted as a confidence level. PyCaret allows you to calibrate the probabilities of a given model through the calibrate_model() function. See an example below

In [ ]:
plot_model(stack_soft, plot='calibration')

By default, calibrate_model() uses the sigmoid method which corresponds to Platt's approach. The other available method is isotonic which is a non-parametric approach. 

In [ ]:
calibrated_rf_isotonic = calibrate_model(stack_soft, method = 'isotonic')

In [ ]:
plot_model(calibrated_rf_isotonic, plot='calibration')

In [ ]:
evaluate_model(calibrated_rf_isotonic)

In [ ]:
predict_model(calibrated_rf_isotonic);

In [ ]:
predict_model(blend_soft);

In [ ]:
final_rf = finalize_model(blend_soft)

In [ ]:
print(final_rf)

In [ ]:
predict_model(final_rf);

In [ ]:
unseen_predictions = predict_model(final_rf, data=data_unseen)
unseen_predictions.head()

In [ ]:
from pycaret.utils import check_metric
check_metric(unseen_predictions['Churn'], unseen_predictions['Label'], metric = 'F1')

In [ ]:
model_paths = "/Users/lavi/Documents/Consulting Work/Upgrad/Alumni Sessions/MLOps/Session01/project/mlops_experiment/models/"
save_model(final_rf, model_paths+'Final_Blend_model_version0.1_12june')

In [ ]:
saved_final_rf = load_model(model_paths+'Final_Blend_model_version0.1_12june')

In [ ]:
new_prediction = predict_model(saved_final_rf, data=data_unseen)

In [ ]:
new_prediction.head()

In [ ]:
from pycaret.utils import check_metric
check_metric(new_prediction['Churn'], new_prediction['Label'], metric = 'Accuracy')

In [ ]:
check_metric(new_prediction['Churn'], new_prediction['Label'], metric = 'F1')

In [ ]:
!mlflow ui